<a href="https://colab.research.google.com/github/JINYUHOON/Smart_Factory_APS_System/blob/main/make_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
plt.rc('font', family='NanumBarunGothic')

In [ ]:
# 제품기본 = pd.read_excel('/content/drive/MyDrive/5조_스마트팩토리/수요,발주예측관리(현재)_20210426.xlsx', sheet_name=2, header=1, index_col=1).drop('Unnamed: 0',axis=1)
# 제품기본.to_csv('/content/drive/MyDrive/5조_스마트팩토리/진유훈/product_basic.csv')

# 원자재기본 = pd.read_excel('/content/drive/MyDrive/5조_스마트팩토리/수요,발주예측관리(현재)_20210426.xlsx', sheet_name=3, header=1, index_col=1).drop('Unnamed: 0',axis=1)
# 원자재기본.to_csv('/content/drive/MyDrive/5조_스마트팩토리/진유훈/material_basic.csv')

# 거래처기본 = pd.read_excel('/content/drive/MyDrive/5조_스마트팩토리/수요,발주예측관리(현재)_20210426.xlsx', sheet_name=4, header=1, index_col=1).drop('Unnamed: 0',axis=1)
# 거래처기본.to_csv('/content/drive/MyDrive/5조_스마트팩토리/진유훈/customer_basic.csv')

# 영업수주 = pd.read_excel('/content/drive/MyDrive/5조_스마트팩토리/수요,발주예측관리(현재)_20210426.xlsx', sheet_name=5, header=1, index_col=1).drop('Unnamed: 0',axis=1)
# 영업수주.to_csv('/content/drive/MyDrive/5조_스마트팩토리/진유훈/business_order.csv')

# 생산LOT = pd.read_excel('/content/drive/MyDrive/5조_스마트팩토리/수요,발주예측관리(현재)_20210426.xlsx', sheet_name=6, header=1, index_col=1).drop('Unnamed: 0',axis=1)
# 생산LOT.to_csv('/content/drive/MyDrive/5조_스마트팩토리/진유훈/LOT_basic.csv')

# 생산레시피 = pd.read_excel('/content/drive/MyDrive/5조_스마트팩토리/수요,발주예측관리(현재)_20210426.xlsx', sheet_name=7, header=1, index_col=1).drop('Unnamed: 0',axis=1)
# 생산레시피.to_csv('/content/drive/MyDrive/5조_스마트팩토리/진유훈/recipe.csv')

# 자재발주 = pd.read_excel('/content/drive/MyDrive/5조_스마트팩토리/수요,발주예측관리(현재)_20210426.xlsx', sheet_name=8, header=1, index_col=1).drop('Unnamed: 0',axis=1)
# 자재발주.to_csv('/content/drive/MyDrive/5조_스마트팩토리/진유훈/material_order.csv')

# 자재납품 = pd.read_excel('/content/drive/MyDrive/5조_스마트팩토리/수요,발주예측관리(현재)_20210426.xlsx', sheet_name=9, header=1, index_col=1).drop('Unnamed: 0',axis=1)
# 자재납품.to_csv('/content/drive/MyDrive/5조_스마트팩토리/진유훈/meaterial_delivery.csv')

In [ ]:
# 제품기본 = pd.read_excel('/content/drive/MyDrive/5조_스마트팩토리/수요,발주예측관리(현재)_20210426.xlsx', sheet_name=2, header=1, index_col=1).drop('Unnamed: 0',axis=1)
# 제품기본.to_csv('/content/drive/MyDrive/5조_스마트팩토리/진유훈/csv테스트.csv')

# 제품코드 => 제품명 통일하기

In [ ]:
제품기본 = pd.read_pickle('/content/drive/MyDrive/5조_스마트팩토리/데이터셋/제품기본.pkl')
제품 = 제품기본[['제품코드','제품명']]
import re

def prod_name(제품명):
    prod_name = re.search('[a-zA-Z]+-*[a-zA-Z]*[\d]*[a-zA-Z]*',제품명).group()
    return prod_name

def region(제품명):
    try:
        region = re.search(r'\([가-힣]+\)',제품명).group().strip('()')
        return region
    except:
        return np.nan

제품['제품명'] = 제품['제품명'].apply(prod_name)
제품['region'] = 제품['제품명'].apply(region)
제품

# 자재발주 분석

In [ ]:
자재발주 = pd.read_pickle('/content/drive/MyDrive/5조_스마트팩토리/데이터셋/자재발주.pkl')
material_order = 자재발주[['발주일자','원자재코드','발주수량']]
material_order

In [ ]:
pd.pivot_table(material_order, index='발주일자', columns='원자재코드',values='발주수량',aggfunc='sum')

In [ ]:
자재발주['원자재코드'].unique()


# 원자재 코드별 원자재명 베이스데이터

In [ ]:
원자재기본 = pd.read_pickle('/content/drive/MyDrive/5조_스마트팩토리/데이터셋/원자재기본.pkl')
원자재 = 원자재기본[['원자재코드(품목번호)','원자재명(품목명)']].rename(columns= {'원자재코드(품목번호)':'원자재코드', '원자재명(품목명)':'원자재명'})

원자재

In [ ]:
생산레시피 = pd.read_pickle('/content/drive/MyDrive/5조_스마트팩토리/데이터셋/생산레시피.pkl')

def org(x):
    a = x.split()
    b = a[0].split('(')
    return b[0]

생산레시피['원자재명'] = 생산레시피['원자재명'].apply(org)
생산레시피['원자재명'].unique()

In [ ]:
생산 = pd.merge(생산레시피, 제품, on='제품코드')
생산

In [ ]:
pivot_lot = pd.pivot_table(생산, index=['LOT번호','제품명'],columns='원자재명',values='투입지시중량', aggfunc='sum').fillna(0)
code_pivot = pivot_lot.groupby('제품명')[list(생산레시피['원자재명'].unique())].mean()
result_data1 = round(code_pivot,0).astype(int)
result_data1

In [ ]:
result_data1.to_csv('/content/drive/MyDrive/5조_스마트팩토리/진유훈/원자재예시.csv')

In [ ]:
bom = 생산레시피.groupby(['생산작업요청일자','LOT번호','제품코드','원자재명','투입지시중량'])[['제품코드','투입지시중량']].sum()
bom.unstack('원자재명').fillna(0)

## 차수는 모든 상품에 대해 날짜 상관없이 ( DATETIME에 영향을 받지 않음.)

# pd.pivot_table(생산레시피, columns=[''])

In [ ]:
/content/drive/MyDrive/5조_스마트팩토리/진유훈/기상청api.ipynb

def prod_name(제품명):
    prod_name = re.search('[a-zA-Z]+-*[a-zA-Z]*[\d]*[a-zA-Z]*',제품명).group()
    return prod_name

제품기본['제품명'] = 제품기본['제품명'].apply(prod_name)
제품기본['제품명'].unique()

In [ ]:
제품기본.reset_index(inplace=True)
product = 제품기본[['제품코드','제품명']]
product



In [ ]:
## 원자재기본에 나와있는 20개원자재 코드<-> 원자재명 
meaterial_data = pd.merge(원자재사용, 원자재, on='원자재코드')
result_data = pd.merge(meaterial_data, product,on='제품코드')
result_data

In [ ]:
# ## 원자재명 바꿔주기

# import re

# def org(x):
#     a = x.split()
#     b = a[0].split('(')
#     return b[0]

# 원자재사용['원자재명'] = 원자재사용['원자재명'].apply(org)
# # 원자재사용

# 원자재사용['원자재명'].unique()

In [ ]:
pd.pivot_table(result_data, index='제품명',columns='원자재명',values='투입지시중량', aggfunc='max').fillna(0)

In [ ]:
test_g = 생산레시피.groupby(['생산작업요청일자','제품코드','원자재코드','원자재명','투입지시중량','투입지시비율'])[['투입지시중량','투입지시비율']].sum()

In [ ]:
len(원자재사용['제품코드'].unique())

In [ ]:
제품 = 제품기본[['제품코드','제품명']]

In [ ]:
판매데이터 = pd.read_csv('/content/drive/MyDrive/5조_스마트팩토리/조현정/판매데이터.csv',index_col=0)
판매데이터

In [ ]:
import re

def prod_name(제품명):
    prod_name = re.search('[a-zA-Z]+-*[a-zA-Z]*[\d]*[a-zA-Z]*',제품명).group()
    return prod_name

def region(제품명):
    try:
        region = re.search(r'\([가-힣]+\)',제품명).group().strip('()')
        return region
    except:
        return np.nan

제품['prod_name'] = 제품['제품명'].apply(prod_name)
제품['region'] = 제품['제품명'].apply(region)

In [ ]:
# 제품 = 제품.drop('제품명',axis=1)
제품

In [ ]:
# 제품[제품['region'].isnull()==True]

In [ ]:
def org(x):
    a = x.split()
    b = a[0].split('(')
    return b[0]

원자재사용['원자재명'] = 원자재사용['원자재명'].apply(org)
원자재사용

In [ ]:
원자재최종 = pd.merge(원자재사용, 제품, on='제품코드')

In [ ]:
원자재최종2 = 원자재최종.sort_values('생산작업요청일자').reset_index(drop=True)
원자재최종2.rename(columns= {'생산작업요청일자':'생산일자',
                             '투입지시중량':'투입중량',
                             '투입지시비율':'투입비율'},inplace=True)
원자재최종2

In [ ]:
원자재최종2['원자재명'].unique()

In [ ]:
원자재최종3 = 원자재최종2[['생산일자','LOT번호','prod_name','원자재명','원자재코드','투입중량','투입비율']]

In [ ]:
원자재최종3.to_pickle('/content/drive/MyDrive/5조_스마트팩토리/진유훈/원자재정리.pkl')

In [ ]:
원자재최종3

In [ ]:
df = pd.pivot_table(원자재최종3, index=['생산일자','LOT번호','prod_name'], columns='원자재명', values='투입비율',aggfunc='sum').fillna(0)
df['sum'] = df.sum(axis=1)


In [ ]:
df[df['sum']<100.0]

In [ ]:
판매데이터 = 판매데이터.sort_values('납기일자').reset_index(drop=True)

In [ ]:
판매데이터